# LangGraph Graph Construction

Build your first LangGraph workflow by connecting nodes and edges.

## Learning Objectives

By the end of this notebook, you will:

1. **Build a graph** by adding nodes and edges
2. **Compile the graph** into an executable application
3. **Test the graph** with a sample query

In [1]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [2]:
# LangGraph and LangChain imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from typing import Literal

print("✅ All imports successful")

✅ All imports successful


In [ ]:
# Mermaid helper for graph visualization
def render_mermaid(diagram_code, width=400):
    """Render Mermaid diagrams using mermaid.ink service."""
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

print("✅ Visualization helper defined")

✅ Visualization helper defined


In [4]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


In [5]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

In [6]:

tools = [currency_converter, emi_calculator]


In [7]:
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

✅ LLM initialized with tools


In [8]:
# Define LLM node
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

print("✅ LLM node defined")

✅ LLM node defined


In [9]:
# Define router function
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

print("✅ Router function defined")

✅ Router function defined


## Build Graph

In [10]:
# Build graph
workflow = StateGraph(MessagesState)

In [11]:
workflow.add_node("llm", call_llm)

In [12]:
workflow.add_node("tools", ToolNode(tools))

In [13]:
workflow.add_edge(START, "llm")

In [14]:
workflow.add_conditional_edges(
    "llm",
    should_continue,
    {"tools": "tools", END: END}
)

In [15]:
workflow.add_edge("tools", "llm")

## Compile Graph

In [16]:
app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


In [17]:
# Visualize graph
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram, width=200)

## Test Graph

In [18]:
# Test the graph
test_state = {
    "messages": [HumanMessage(content="Convert 100 USD to EUR")]
}

In [19]:
result = app.invoke(test_state)

In [20]:
print(f"Total messages: {len(result['messages'])}")

Total messages: 4


In [21]:
type(result)

dict

In [22]:
from pprint import pprint

In [23]:
pprint(result)

{'messages': [HumanMessage(content='Convert 100 USD to EUR', additional_kwargs={}, response_metadata={}, id='8d069d27-1368-440e-a8ff-e42f6b9f007e'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_converter', 'arguments': '{"amount": 100, "to_currency": "EUR", "from_currency": "USD"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c2e05-2f44-7821-902a-6966e03785e3-0', tool_calls=[{'name': 'currency_converter', 'args': {'amount': 100, 'to_currency': 'EUR', 'from_currency': 'USD'}, 'id': 'e37edf0d-3843-47c3-8e1e-52c10e0e6867', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 136, 'output_tokens': 13, 'total_tokens': 149, 'input_token_details': {'cache_read': 0}}),
              ToolMessage(content='Conversion Result:\n  100.00 USD = 92.00 EUR\n  Exchange Rate: 1 USD = 0.9200 EUR', name='currency_converter', id='6

In [24]:
print(f"\nFinal Response:\n{result['messages'][-1].content}")


Final Response:
100 USD is approximately equal to 92.00 EUR.


## Conclusion

### What You've Accomplished

✅ Built a graph with nodes and edges

✅ Compiled the graph into an application

✅ Tested the graph with a query

### Next Steps

Continue to **Notebook 06: Single Tool Execution** to understand the message flow.